### 📥 Leitura da Base Bruta (Excel)

Este trecho localiza automaticamente a raiz do projeto e constrói o caminho até o arquivo Excel original (BASE DE DADOS PEDE 2024 - DATATHON.xlsx), armazenado em data/raw.

Após validar que o arquivo existe, utilizamos pd.read_excel(..., sheet_name=None) para carregar todas as abas do Excel em um dicionário, onde cada chave representa um ano da base.

Essa é a etapa inicial de ingestão dos dados brutos antes do pré-processamento.

In [1]:
from pathlib import Path
import pandas as pd

# Descobre a raiz do projeto (assume que o notebook está em notbooks/)
try:
    ROOT = Path(__file__).resolve().parents[1]
except NameError:
    # __file__ não existe em notebooks; usa o cwd como base
    ROOT = Path.cwd().resolve().parent

data_path = ROOT / 'data' / 'raw' / 'BASE DE DADOS PEDE 2024 - DATATHON.xlsx'

if not data_path.exists():
    raise FileNotFoundError(f"Arquivo não encontrado: {data_path}")

sheets = pd.read_excel(data_path, sheet_name=None)



### 🔗 Consolidação das Abas em um Único Dataset

Este trecho percorre todas as abas do arquivo Excel, extrai o ano presente no nome de cada aba (usando expressão regular) e cria a coluna ANO correspondente.

Em seguida, todas as abas são concatenadas em um único DataFrame (df_all), unificando os dados de diferentes anos em uma base consolidada para análise e pré-processamento.

In [2]:
import re

dfs = []

for nome_aba, df in sheets.items():
    df = df.copy()
    
    # extrai o ano (4 dígitos)
    ano = re.search(r"\d{4}", nome_aba)
    
    if not ano:
        raise ValueError(f"Aba sem ano identificável: {nome_aba}")
    
    df["ANO"] = int(ano.group())
    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)

### 📋 Inspeção da Base Consolidada

O comando df_all.info() exibe um resumo estrutural do dataset após a consolidação das abas.

Essa verificação inicial é essencial para entender a estrutura da base bruta antes de iniciar o pré-processamento.

In [3]:
df_all.info()

<class 'pandas.DataFrame'>
RangeIndex: 3030 entries, 0 to 3029
Data columns (total 64 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   RA                     3030 non-null   str    
 1   Fase                   3030 non-null   object 
 2   Turma                  3030 non-null   object 
 3   Nome                   860 non-null    str    
 4   Ano nasc               860 non-null    float64
 5   Idade 22               860 non-null    float64
 6   Gênero                 3030 non-null   str    
 7   Ano ingresso           3030 non-null   int64  
 8   Instituição de ensino  3029 non-null   str    
 9   Pedra 20               754 non-null    str    
 10  Pedra 21               1061 non-null   str    
 11  Pedra 22               1932 non-null   str    
 12  INDE 22                1932 non-null   float64
 13  Cg                     860 non-null    float64
 14  Cf                     860 non-null    float64
 15  Ct             

### 🔄 Ordenação Temporal dos Dados

Este trecho ordena o dataset consolidado (df_all) pelo identificador do aluno (RA) e pelo ANO.

Isso garante que o histórico de cada estudante esteja organizado cronologicamente, facilitando análises temporais e a construção correta de variáveis como presença no ano seguinte ou abandono.

In [4]:
df_all = df_all.sort_values(
    by=["RA", "ANO"]
).reset_index(drop=True)

### 📊 Análise de Permanência dos Alunos

Este comando calcula quantos anos distintos cada aluno (RA) aparece na base e, em seguida, mostra a distribuição dessa contagem.

Essa análise é importante para entender retenção, evasão e consistência do histórico temporal antes da modelagem.

In [5]:
df_all.groupby("RA")["ANO"].nunique().value_counts()

ANO
1    760
3    468
2    433
Name: count, dtype: int64

### 📑 Tamanho de Cada Aba

Este trecho percorre todas as abas do arquivo Excel e imprime o nome de cada uma junto com seu shape (linhas, colunas).
É uma etapa importante de validação inicial da base bruta.

In [6]:
for nome_aba, df in sheets.items():
    print(nome_aba, df.shape)

PEDE2022 (860, 42)
PEDE2023 (1014, 48)
PEDE2024 (1156, 50)


### 🔍 Comparação de Colunas Entre os Anos

Este trecho compara as colunas das abas PEDE2022, PEDE2023 e PEDE2024, identificando quais variáveis foram adicionadas ou alteradas entre os anos.

Essa análise é fundamental para garantir consistência estrutural antes de unificar e tratar os dados para modelagem.

In [7]:
cols_2022 = set(sheets["PEDE2022"].columns)
cols_2023 = set(sheets["PEDE2023"].columns)
cols_2024 = set(sheets["PEDE2024"].columns)

print("2023 - 2022:", cols_2023 - cols_2022)
print("2024 - 2023:", cols_2024 - cols_2023)
print("2024 - 2022:", cols_2024 - cols_2022)

2023 - 2022: {'Fase Ideal', 'Mat', 'INDE 2023', 'Destaque IPV.1', 'Nome Anonimizado', 'Idade', 'IPP', 'Ing', 'Defasagem', 'Pedra 2023', 'INDE 23', 'Data de Nasc', 'Por', 'Pedra 23'}
2024 - 2023: {'Ativo/ Inativo.1', 'INDE 2024', 'Pedra 2024', 'Avaliador6', 'Ativo/ Inativo', 'Escola', 'Avaliador5'}
2024 - 2022: {'Fase Ideal', 'INDE 2024', 'Mat', 'Pedra 2024', 'Ativo/ Inativo.1', 'Avaliador6', 'Nome Anonimizado', 'Idade', 'IPP', 'Ativo/ Inativo', 'Ing', 'Defasagem', 'Escola', 'INDE 23', 'Data de Nasc', 'Por', 'Pedra 23', 'Avaliador5'}


### 🧹Funções de Padronização e Parsing

Este bloco define funções utilitárias para limpar e padronizar os dados brutos antes do pré-processamento.

In [8]:
import pandas as pd
import numpy as np
import re
import unicodedata

def normalize_col(col: str) -> str:
    col = str(col).strip()
    col = unicodedata.normalize("NFKD", col).encode("ascii", "ignore").decode("ascii")
    col = col.lower()
    col = re.sub(r"[\/\.\-]+", "_", col)
    col = re.sub(r"\s+", "_", col)
    col = re.sub(r"[^a-z0-9_]", "", col)
    col = re.sub(r"_+", "_", col).strip("_")
    return col

def coalesce_cols(df: pd.DataFrame, candidates: list[str]) -> pd.Series:
    cols = [c for c in candidates if c in df.columns]
    if not cols:
        return pd.Series([np.nan] * len(df), index=df.index)
    return df[cols].bfill(axis=1).iloc[:, 0]

def parse_idade(valor):
    """
    Limpeza: transforma idade em número.
    - se já for número plausível: mantém
    - se vier como data tipo 08/01/1900 (erro Excel): usa o 'dia' (8)
    """
    if pd.isna(valor):
        return np.nan

    # numérico direto
    if isinstance(valor, (int, float, np.integer, np.floating)):
        return float(valor) if 0 < float(valor) < 120 else np.nan

    # tentar converter strings/datas
    dt = pd.to_datetime(valor, errors="coerce", dayfirst=True)
    if pd.notna(dt) and dt.year == 1900:
        return float(dt.day)

    # tentar número vindo como string (ex: "8")
    try:
        v = float(str(valor).replace(",", "."))
        return v if 0 < v < 120 else np.nan
    except Exception:
        return np.nan


def parse_fase_int(valor):
    """
    Converte FASE / FASE_IDEAL para inteiro seguindo regras:

    1) Se vier só número: 2, 2.0, "2", "2.0" -> 2
    2) Se vier número + letra: "1A", "2B", "3a" -> número
    3) Se contiver "alfa" ou "alpha" (qualquer caixa) -> 0
    4) Se contiver "fase X" (FASE 1..8, com variações) -> X
    Retorna pd.NA se não conseguir interpretar.
    """
    # evita erro com NA do pandas
    try:
        if valor != valor:
            return pd.NA
    except Exception:
        pass

    # numérico puro
    if isinstance(valor, int):
        return valor

    if isinstance(valor, float):
        if valor.is_integer():
            return int(valor)
        return pd.NA

    s = str(valor).strip()
    if s == "" or s.lower() in {"nan", "none"}:
        return pd.NA

    sl = s.lower()

    # 1) só número ("2")
    if sl.isdigit():
        return int(sl)

    # 2) número com .0 ("2.0", "2.00")
    if sl.replace(".", "", 1).isdigit() and sl.endswith(".0"):
        try:
            return int(float(sl))
        except Exception:
            return pd.NA

    # 3) número + letra ("1A", "2b")
    if len(sl) >= 2 and sl[:-1].isdigit() and sl[-1].isalpha():
        return int(sl[:-1])

    # 4) alfa / alpha
    if "alfa" in sl or "alpha" in sl:
        return 0

    # 5) fase X (ex: "FASE 2", "fase-3", "fase_4")
    if "fase" in sl:
        num = ""
        for c in sl:
            if c.isdigit():
                num += c
            elif num:
                break
        if num:
            return int(num)

    return pd.NA
  


### 🏗️ Construção da Base de Features a partir dos Dados Brutos

A função build_features_base_raw consolida o df_all (abas unificadas) em uma base padronizada de features essenciais para modelagem.

Ao final, retorna apenas as colunas definidas em FEATURES_BASE_RAW, formando a base “crua organizada” para as próximas etapas do pipeline.

In [11]:
def build_features_base_raw(df_all: pd.DataFrame) -> pd.DataFrame:
    df = df_all.copy()
    df.columns = [normalize_col(c) for c in df.columns]

    # Ajuste se seus nomes normalizados forem diferentes
    if "ra" not in df.columns:
        raise ValueError("Não encontrei coluna RA (após normalização).")
    if "ano" not in df.columns:
        raise ValueError("Não encontrei coluna ANO (após normalização).")

    # candidatos (apenas mapeamento/organização)
    cand = {
        "idade": ["idade", "idade_22", "idade_2022", "idade_2023", "idade_2024", "idade_aluno"],
        "fase": ["fase", "fase_2022", "fase_2023", "fase_2024"],
        #"fase_ideal": ["fase_ideal", "nivel_ideal", "nivel_ideal_2021", "nivel_ideal_2022"],
        "defasagem": ["defasagem", "defasagem_2021", "defasagem_2022", "defasagem_2023", "defasagem_2024", "defas"],

        "iaa": ["iaa", "iaa_2022", "iaa_2023", "iaa_2024"],
        "ieg": ["ieg", "ieg_2022", "ieg_2023", "ieg_2024"],
        "ida": ["ida", "ida_2022", "ida_2023", "ida_2024"],
        "ian": ["ian", "ian_2022", "ian_2023", "ian_2024"],
        "ips": ["ips", "ips_2022", "ips_2023", "ips_2024"],
        "ipv": ["ipv", "ipv_2022", "ipv_2023", "ipv_2024"],

        "nota_mat": ["nota_mat", "nota_mat_2022", "nota_mat_2023", "nota_mat_2024", "mat", "matem"],
        "nota_por": ["nota_por", "nota_port", "nota_por_2022", "nota_por_2023", "nota_por_2024", "por", "portug"],
    }

    out = pd.DataFrame(index=df.index)
    out["RA"] = df["ra"].astype(str).str.strip()
    out["ANO"] = pd.to_numeric(df["ano"], errors="coerce")

    # IDADE: só limpeza de formato (sem inventar valor)
    idade_raw = coalesce_cols(df, cand["idade"])
    out["IDADE"] = idade_raw.apply(parse_idade)

    out["FASE"] = coalesce_cols(df, cand["fase"]).apply(parse_fase_int).astype("Int64")
    #out["FASE_IDEAL"] = coalesce_cols(df, cand["fase_ideal"]).apply(parse_fase_int).astype("Int64") 

    # target original (se for usar)
    out["DEFASAGEM"] = pd.to_numeric(coalesce_cols(df, cand["defasagem"]), errors="coerce")

    # indicadores
    for k in ["iaa","ieg","ida","ian","ips","ipv"]:
        out[k.upper()] = pd.to_numeric(coalesce_cols(df, cand[k]), errors="coerce")

    # notas
    out["NOTA_MAT"] = pd.to_numeric(coalesce_cols(df, cand["nota_mat"]), errors="coerce")
    out["NOTA_POR"] = pd.to_numeric(coalesce_cols(df, cand["nota_por"]), errors="coerce")

    # limpeza mínima estrutural
    out = out.dropna(subset=["RA", "ANO"])

    # clipping leve (não cria info nova; só corrige ruído numérico)
    for col in ["IAA","IEG","IDA","IAN","IPS","IPV","NOTA_MAT","NOTA_POR","NOTA_ING"]:
        if col in out.columns:
            out[col] = out[col].clip(lower=0, upper=10)

    # *** SEM DEFASAGEM_LAG AQUI ***
    # *** SEM FLAGS/DELTAS AQUI ***

    FEATURES_BASE_RAW = [
        "RA","ANO","IDADE","FASE",#"FASE_IDEAL",
        "IAA","IEG","IDA","IAN","IPS","IPV",
        "NOTA_MAT","NOTA_POR",
        "DEFASAGEM"
    ]
    return out[FEATURES_BASE_RAW]

### 📦 Geração da Base Estruturada

Aqui executamos a função build_features_base_raw para criar df_base, que contém as principais variáveis padronizadas e prontas para pré-processamento.

Em seguida:

 - df_base.shape confirma o número de linhas e colunas da nova base.

 - df_base.head() exibe as primeiras linhas para validação visual da estrutura e consistência dos dados.

In [12]:
df_base = build_features_base_raw(df_all)

df_base.shape, df_base.head()

((3030, 13),
        RA   ANO  IDADE  FASE  IAA  IEG  IDA   IAN  IPS    IPV  NOTA_MAT  \
 0    RA-1  2022   19.0     7  8.3  4.1  4.0   5.0  5.6  7.278       2.7   
 1    RA-1  2023   20.0     8  NaN  NaN  NaN  10.0  NaN    NaN       NaN   
 2    RA-1  2024   21.0     8  NaN  0.0  NaN  10.0  NaN    NaN       NaN   
 3   RA-10  2022   18.0     7  8.3  5.2  4.1   5.0  5.0  7.056       3.3   
 4  RA-100  2022   13.0     4  8.8  7.8  7.6  10.0  5.0  7.250       7.0   
 
    NOTA_POR  DEFASAGEM  
 0       3.5       -1.0  
 1       NaN        0.0  
 2       NaN        0.0  
 3       2.6       -1.0  
 4       7.8        1.0  )

### 📊 Análise de Nulos por Ano

Este código calcula, para cada ANO, o percentual de valores ausentes em cada coluna, ordena do maior para o menor e exibe as 20 variáveis com mais nulos.

Isso ajuda a identificar diferenças na qualidade ou disponibilidade dos dados entre os anos, orientando decisões de tratamento e imputação.

In [13]:
(df_base
 .groupby("ANO")
 .apply(lambda x: x.isna().mean().sort_values(ascending=False).head(20))
)

ANO            
2022  NOTA_MAT     0.002326
      NOTA_POR     0.002326
      RA           0.000000
      IDADE        0.000000
      IAA          0.000000
      FASE         0.000000
      IEG          0.000000
      IDA          0.000000
      IPS          0.000000
      IAN          0.000000
      IPV          0.000000
      DEFASAGEM    0.000000
2023  NOTA_MAT     0.075937
      NOTA_POR     0.075937
      IDA          0.075937
      IEG          0.074951
      IPV          0.074951
      IPS          0.068047
      IAA          0.062130
      RA           0.000000
      FASE         0.000000
      IDADE        0.000000
      IAN          0.000000
      DEFASAGEM    0.000000
2024  NOTA_POR     0.091696
      NOTA_MAT     0.090830
      IAA          0.088235
      IPS          0.088235
      IPV          0.088235
      IDA          0.087370
      RA           0.000000
      IDADE        0.000000
      FASE         0.000000
      IEG          0.000000
      IAN          0.000000
    

### 📈 Estatísticas Descritivas dos Indicadores

Este comando gera estatísticas descritivas (count, média, desvio padrão, mínimo, quartis e máximo) para os principais indicadores e notas. Servindo como validação antes das próximas etapas do pipeline.

In [ ]:
df_base[["IAA","IEG","IDA","IAN","IPS","IPV","NOTA_MAT","NOTA_POR"]].describe()

,IAA,IEG,IDA,IAN,IPS,IPV,NOTA_MAT,NOTA_POR
count,2865.000000,2954.000000,2852.000000,3030.000000,2859.000000,2852.000000,2846.000000,2845.000000
mean,7.918140,7.945696,6.375964,7.179043,6.287129,7.545385,6.161595,6.430668
std,2.626142,2.152281,1.956637,2.535266,1.792491,1.084140,2.398341,2.138916
min,0.000000,0.000000,0.000000,2.500000,2.500000,2.500000,0.000000,0.000000
25%,7.900000,7.300000,5.100000,5.000000,5.020000,6.984000,4.800000,5.200000
50%,8.751000,8.600000,6.666667,5.000000,7.500000,7.583000,6.300000,6.700000
75%,9.500000,9.400000,7.833333,10.000000,7.510000,8.255000,8.000000,8.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


### 📊 Histórico de Participação dos Alunos

Este comando calcula quantos anos distintos cada aluno (RA) aparece na base e depois mostra a distribuição dessa contagem.

Com isso, conseguimos identificar:

- Quantos alunos participaram apenas 1 ano,

- Quantos permaneceram por 2 anos,

- Quantos têm histórico completo (3 anos).

Essa informação é importante para análises temporais e para a definição correta do target de abandono.

In [15]:
df_base.groupby("RA")["ANO"].nunique().value_counts()


ANO
1    760
3    468
2    433
Name: count, dtype: int64

### 💾 Salvando a Base Refinada

Este trecho exporta df_base no formato Parquet, armazenando-o em data/refined.

Utilizamos o engine pyarrow com compressão snappy, garantindo melhor performance e eficiência de armazenamento.

Essa etapa consolida a base organizada e pronta para ser utilizada nas próximas fases do pipeline de modelagem.

In [14]:
# Salvar base consolidada em Parquet
# Ajustar tipos problemáticos antes de exportar
output_path = '../data/refined/df_base.parquet'
df_base.to_parquet(output_path, index=False, engine='pyarrow', compression='snappy')
print(f'Arquivo Parquet salvo em {output_path}')


Arquivo Parquet salvo em ../data/refined/df_base.parquet
